# Snuffelfiets kwartaalrapportages

In [ ]:
## Google Colab pre-amble.
try:
    from google.colab import drive, files
    msg = 'Running in colab.'
except:
    msg = 'Running local.'
else:

    drive.mount('/content/gdrive')
    
    !pip install -q condacolab
    import condacolab
    condacolab.install()

    !git clone https://github.com/MilieuCentrumUtrecht/Snuffelfiets.git

    %cd Snuffelfiets

    !conda develop .

    !mamba env update -n base -f environment.yml

print(msg)


## Settings.

In [ ]:
# Imports and convenience functions.

from pathlib import Path

import numpy as np
import pandas as pd

from snuffelfiets import inlezen, opschonen, analyse, plotting


def get_period_info(period_spec, year, quarter, month):
    """Return an id-string and the months for the chosen period."""

    quarters = {
        'Q1': [1, 2, 3], 
        'Q2': [4, 5, 6], 
        'Q3': [7, 8, 9], 
        'Q4': [10, 11, 12], 
    }
    periods = {
        'jaar': {
            'period_id': f'{year}',
            'months': list(range(1, 13)),
        },
        'kwartaal': {
            'period_id': f'{year}_{quarter}',
            'months': quarters[quarter],
        },
        'maand': {
            'period_id': f'{year}_{month:02d}',
            'months': [month],
        }
    }
    period_id = periods[period_spec]['period_id']
    months = periods[period_spec]['months']

    return period_id, months


def get_center():
    """Return coordinates for the center of provincie Utrecht."""

    # coordinaten van de uiterste punten van de provincie Utrecht
    b = {
        'N': [52.303634, 5.013507],
        'Z':  [51.858631, 5.040462],
        'O':  [51.954780, 5.627990],
        'W':  [52.226808, 4.794457],
    }

    # het berekende middelpunt van de provincie Utrecht
    center = {
        'lat': b['Z'][0] + 0.5 * (b['N'][0] - b['Z'][0]),
        'lon': b['W'][1] + 0.5 * (b['O'][1] - b['W'][1]),
    }

    return center


def get_layers(data_directory):
    """Import Utrecht province and township polygons"""

    # Download the polygons.
    filepaths = plotting.download_borders_utrecht(data_directory)

    # Extract the relevant polygons.
    provincies, gemeenten = plotting.get_borders_utrecht(data_directory, *filepaths)

    # Enter into dictionary in the mapbox_layers format.
    mapbox_layers = [{
        "name": "Gemeenten",
        "below": 'traces',
        "sourcetype": "geojson",
        "type": "line",
        "color": "gray",
        "source": gemeenten,
        }, 
        {
        "name": "Provincies",
        "below": 'traces',
        "sourcetype": "geojson",
        "type": "line",
        "color": "red",
        "source": provincies,
        }]

    return mapbox_layers


In [ ]:
# API settings.
api_key = ''                    # de API key

# File system settings.
prefix = 'api_gegevens'         # de prefix voor de csv-datafiles; default format <prefix>_<yyyy>_<mm>.csv
data_directory = Path(
    Path('~').expanduser(),
    'Documents',
    'MCUdataclub',
    'Snuffelfiets_data',
    'rapportage',
    )                           # het pad naar de folder met de data; dit wordt ook de parent folder van de output
Path.mkdir(data_directory, parents=True, exist_ok=True)

# Date range settings.
period_spec = 'kwartaal'        # de periode waar het rapport over gaat: 'maand', 'kwartaal' of 'jaar'?
year = 2024                     # het jaar waar het rapport over gaat
quarter = 'Q2'                  # het kwartaal waar het rapport over gaat: 'Q1', 'Q2', 'Q3', 'Q4'
month = 6                       # de maand waar het rapport over gaat: 1, 2, 3, ..., 10, 11, 12


# Data processing settings.
error_code_selection = []       # te verwijderen error codes; [] => behoud alleen error_code=0
rit_splitter_interval = 1800    # het interval tussen ritten (s)
ritfilters = dict(
    min_measurements=2,             # #
    max_duration=360,               # minutes
    max_distance=200,               # kilometers
    min_average_speed=1,            # km/h
    max_average_speed=35,           # km/h
    )                           # criteria waaraan ritten moeten voldoen
threshold_pm2_5 = 100           # de cutoff value voor geldige PM2.5 waardes


# Hexbin plot settings.
mapbox_center = get_center()    # het middelpunt van de provincie Utrecht
mapbox_extent = 1               # de breedte rondom de mapbox_center [deg lat/lon]; datapunten buiten deze breedte/lengtegraden worden verwijderd
hexagon_size = 0.010            # de grootte van de hexagons in de hexbin plot
hexbin_args = {
    'agg_func': np.nanmean,
    'color_continuous_scale': plotting.discrete_colorscale(),
    'range_color': [0, threshold_pm2_5],
    'min_count': 2,
    'animation_frame': None,
    'width': 1920,
    'height': 1080,
    'opacity': 1.0,
    'zoom': 10,
    'center': mapbox_center,
    }                           # parameters for creating the hexbin plot
mapbox_layers = get_layers(data_directory)
layout_args = {
    'mapbox_style': 'open-street-map',
    'coloraxis_showscale': False,
    'mapbox_layers': mapbox_layers,
    }                           # parameters for the layout of the hexbin plot

# Directories.
period_id, months = get_period_info(period_spec, year, quarter, month)
output_directory = Path(data_directory, period_id)
output_directory.mkdir(parents=True, exist_ok=True)

print(f'Analysing {period_spec} {period_id}; writing output to {output_directory}.')


## Read data from monthly CSV's.


In [ ]:
# Read the data for the selected period (in monthly chunks).

dfs = []

for m in months:

    filename = f'{prefix}_{year}-{m:02d}.csv'
    p = Path(data_directory, filename)

    try:

        # try to read the data from saved csv's
        df = pd.read_csv(p)

    except FileNotFoundError:

        # download the data if the file does not exist
        inlezen.monthly_csv_dump(api_key, year, m, data_directory, prefix)

        df = pd.read_csv(p)

    dfs.append(df)

df = pd.concat(dfs)

print(f'Read {df.shape[0]} measurements.')


## Data preparation

In [ ]:
# Get some insight in the error modes present in the dataset.
opschonen.analyse_errors(df)


In [ ]:
# Drop the errors.
df = opschonen.verwijder_errors(df, error_codes=error_code_selection)


In [ ]:
# Convert timestamps to datetime objects and add dt columns.
df = analyse.bewerk_timestamp(df, split=True)


In [ ]:
# Split measurements into rides and add cycle stat columns.
df = analyse.split_in_ritten(df, t_seconden=rit_splitter_interval)


In [ ]:
# Filter the rides.
df = analyse.filter_ritten(df, **ritfilters)


## Summary Snuffelfiets statistics.

In [ ]:
def printfun(period, sumstats):

    print(f'\n==== {period} ====\n')

    print(f"Aantal fietsers: {sumstats['fietsers']['N']}\n")

    print(f"{' ':20} {'totaal':12} {'gemiddeld':12} {'topper':12}")
    print(f'-' * 56)
    stat = 'uren'
    print(f"FIETSTIJD [uur]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'afstand'
    print(f"AFSTAND    [km]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'ritten'
    print(f"RITTEN      [#]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")


In [ ]:
# Print the summary statistics for the period.
sumstats = analyse.summary_stats(df)
printfun(period_id, sumstats)


In [ ]:
# Print the summary statistics for the months in the quarter.
for m, dfm in df.groupby('month'):
    sumstats = analyse.summary_stats(dfm)
    printfun(f'{year}{m:02d}', sumstats)


## Air quality evaluation

In [ ]:
# Plot categorical hist of PM2.5 values.
def plot_hbar_cat(df, bins, labels, title=''):
    df[f'{var}_cat'] = pd.cut(df[var], bins=bins, labels=labels)
    ax = df[['entity_id', f'{var}_cat']].groupby(f'{var}_cat', observed=False).count().plot.barh(stacked=True, legend=False)
    ax.invert_yaxis()
    ax.axes.get_yaxis().get_label().set_visible(False)
    ax.axes.get_xaxis().set_label_text("Measurement count")
    ax.set_title(title)

var = 'pm2_5'
bins = [0, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
labels = [0, 1, 2, 5, 10, 20, 50, 100, 200, 500]
plot_hbar_cat(df, bins, labels, title='PM2.5')


In [ ]:
# Limit the PM2.5 values. FIXME: we need a better and validated method
df['pm2_5'][df['pm2_5'] >= threshold_pm2_5] = np.nan

# Plot the histogram.
df['pm2_5'].hist(bins=200, grid=False)


In [ ]:
## Hexbin plots of PM2.5

# Remove datapoints outside of the map area 
#   because it would take a very long time to process large areas.
#   TODO: doe dit op ritniveau (verwijderen ritten deels of geheel buiten de target area)
latlon = {
    'latitude': {'center': mapbox_center['lat'], 'extent': mapbox_extent},
    'longitude': {'center': mapbox_center['lon'], 'extent': mapbox_extent},
}
df = opschonen.filter_lat_lon(df, latlon)

# Plot the data for the period.
hexbin_args['title'] = period_id
fig = plotting.hexbin_mapbox(df, hexagon_size, hexbin_args, layout_args)

# Save image
filestem = f'{period_id}_hexbin'
output_stem = Path(output_directory, filestem)
fig.write_html(f"{output_stem}.html")
fig.write_image(f"{output_stem}.png")
fig.write_image(f"{output_stem}.pdf")

# Plot the data for each month.
for m, dfm in df.groupby('month'):

    yyyymm = f'{year}{m:02d}'
    hexbin_args['title'] = yyyymm
    fig = plotting.hexbin_mapbox(dfm, hexagon_size, hexbin_args, layout_args)

    # Save image
    filestem = f'{yyyymm}_hexbin'
    output_stem = Path(output_directory, filestem)
    fig.write_html(f"{output_stem}.html")
    fig.write_image(f"{output_stem}.png")
    fig.write_image(f"{output_stem}.pdf")


## Uitgelicht 2023-Q1: Fijnstof en regen.

In [ ]:
# Specify the date range.
import calendar
lastday = calendar.monthrange(year, months[-1])[1]
dt_min = f'{year}-{months[0]}-1 00:00:00'
dt_max = f'{year}-{months[-1]}-{lastday} 23:59:59'

variables = {
    'RH': 'Etmaalsom van de neerslag [mm]',
    'TG': 'Etmaalgem van de temperatuur [grC]',
    'FG': 'Etmaalgem van de windsnelheid [m/s]',
}

# Import the weather data.
dfr = analyse.import_knmi_data(dt_min, dt_max, interval='dag', stations=[260], variables=variables.keys())
dfr.RH = dfr.RH.values / 10  # RH: Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)
dfr.TG = dfr.TG.values / 10  # TG: Etmaalgem van de temperatuur (in 0.1 gr C)
dfr.FG = dfr.FG.values / 10  # FG: Etmaalgem van de windsnelheid (in 0.1 m/s)


In [ ]:
# Print the monthly pm2.5 average.
df.loc[: , ['month', 'pm2_5']].groupby('month').mean()


In [ ]:
# Print the monthly rainfall sum.
dfr['month'] = dfr.index.month
dfr.loc[: , ['month'] + list(variables.keys())].groupby('month').sum()


In [ ]:
# Calculate the mean daily PM2.5 value.
sfday = df.loc[:, ['pm2_5', 'day', 'month', 'year']].groupby(['day', 'month', 'year']).mean()
# Add to the weather data frame.
dfr['pm2_5'] = np.array(sfday['pm2_5'])


In [ ]:
# Plot the data.
import matplotlib.pyplot as plt

def plot_knmi_trace(dfr, colname, label=''):

    fig, ax1 = plt.subplots(dpi=300)
    ax2 = ax1.twinx()

    ax1.plot(dfr.index, dfr.pm2_5, c='C1')
    ax2.plot(dfr.index, dfr[colname], c='C0')

    ax1.set_ylabel('PM2.5 [ug/m3]', color='C1')
    ax2.set_ylabel(label, color='C0')

    ax1.plot([dfr.index[0], dfr.index[-1]], [25, 25], c='C1',linestyle='--')
    plt.text(0.86, 0.56, 'norm', c='C1', transform=ax1.transAxes)
    ax1.xaxis.set_tick_params(rotation=45)
    plt.grid(linestyle=':')


In [ ]:
for colname, label in variables.items():
    plot_knmi_trace(dfr, colname, label)

## Uitgelicht 2023-Q2: Fietsgedrag.


In [ ]:
# Total number of rides.
Nritten = len(np.unique(df.loc[:,['rit_id']]))


In [ ]:
p_dict = {'jaar': ['elk', 'dit'], 'kwartaal': ['elk', 'dit'], 'maand': ['elke', 'deze']}
print(f"We geven jullie {p_dict[period_spec][0]} {period_spec} hiernaast de Toppers, maar niet elk ritje kan uitzonderlijk zijn.")
print(f"We zoomen dit keer in op de {Nritten} ritten die er gemaakt zijn {p_dict[period_spec][0]} {period_spec}.")
print("Hoe ziet zo’n ritje er over het algemeen uit en waar fietsen jullie het meest?")


In [ ]:
# Get aggregated rides dataframe.
options = {
    'rit_id': ['count'],
    'duur':['sum'],
    'afstand': ['sum'],
    'snelheid': ['mean'],
    }
df_ritten = df.groupby(['entity_id', 'rit_id']).agg(options)
df_ritten = df_ritten.reset_index(level=['entity_id', 'rit_id'])
cols = ['entity_id', 'rit_id', 'aantal_waarn', 'duur', 'afstand', 'snelheid_mean']
df_ritten = df_ritten.set_axis(cols, axis=1)
df_ritten


In [ ]:
# Perform unit conversions.
df_ritten['duur_h'] = df_ritten['duur'].dt.total_seconds() / 3600  # h
df_ritten['afstand_km'] = df_ritten['afstand'] / 1000  # km
df_ritten['snelheid_mean_kph'] = df_ritten['snelheid_mean'] / 1000 * 3600 # kph


In [ ]:
# Plot simple histograms.
df_ritten[['duur_h', 'afstand_km', 'snelheid_mean_kph']].hist(bins=20, grid=False, figsize=(9, 3), layout=(1, 3))


In [ ]:
# Duur van de ritten.
Nritten_lang = int(np.sum(df_ritten[['duur_h']] > 4))
Nritten_kort = int(np.sum(df_ritten[['duur_h']] < 5/60))
Pritten_kort = int((Nritten_kort / Nritten) * 100)


In [ ]:
print(f"De meeste ritten zijn korter dan een uur.")
print(f"Een klein deel ({Pritten_kort}%) is zelfs korter dan 5 minuten.")
print(f"{Nritten_lang} ritten waren langer dan 4 uur.")


In [ ]:
# Afstand van de ritten.
Nritten_dichtbij = int(np.sum(df_ritten[['afstand_km']] < 5))
Pritten_dichtbij = int((Nritten_dichtbij / Nritten) * 100)

far = df_ritten.loc[df_ritten['afstand_km'] > 20]
far.describe()


In [ ]:
print(f"Lokale ritjes lijken dan ook het populairst: meer dan de helft ({Pritten_dichtbij}%) is korter dan 5 kilometer.")


In [ ]:
# Snelheid van de ritten.
fast = df_ritten.loc[df_ritten['snelheid_mean_kph'] > 25]
Nritten_fast = int(np.sum(df_ritten[['duur_h']] > 4))
fast_and_far = fast.loc[fast['afstand_km'] > 20]
Pfast_and_far = int((fast_and_far.shape[0] / fast.shape[0]) * 100)

slow = df_ritten.loc[df_ritten['snelheid_mean_kph'] < 5]


In [ ]:
print("Er is best wat variatie in de snelheid die gefietst wordt.")
print(f"{fast.shape[0]} ritten bereikten een gemiddelde snelheid hoger dan 25 km/u.")
print(f"Het merendeel ({Pfast_and_far}%) hiervan waren langere ritten van meer dan 20 km!")
print(f"Relaxte ritjes (< 5 km/u) waren er ook genoeg. In {slow.shape[0]} ritjes werd gemiddeld {np.mean(slow['afstand_km']):.1f} km afgelegd.")


In [ ]:
# Plot categorical graphs.

p_dict = {'jaar': [0, 10000, 2500], 'kwartaal': [0, 2500, 1000], 'maand': [0, 1000, 250]}

def plot_hbar_cat(ax, df, bins, labels, title='', xslc=p_dict[period_spec]):
    df[f'{var}_cat'] = pd.cut(df[var], bins=bins, labels=labels)
    ax = df[['entity_id', f'{var}_cat']].groupby(f'{var}_cat', observed=False).count().plot.barh(stacked=True, legend=False, ax=ax)
    ax.invert_yaxis()
    ax.axes.get_yaxis().get_label().set_visible(False)
    # ax.axes.get_xaxis().set_label_text("Aantal ritten")
    ax.set_xlim(xslc[0], xslc[1])
    ax.set_xticks(range(xslc[0], xslc[1], xslc[2]))
    ax.set_title(title)

import matplotlib.pyplot as plt
fig, axs = plt.subplots(3, 1, figsize=(3, 9), sharex=True)

var = 'duur_h'
bins = [0.00, 0.0833, 0.25, 0.50, 1.00, 4.00, np.inf]
labels = ['< 5 min', '5-15 min', '15-30 min', '30-60 min', '1-4 uur', '> 4 uur']
plot_hbar_cat(axs[0], df_ritten, bins, labels, title='Duur van de ritten')

var = 'afstand_km'
bins = [0.00, 0.25, 1.00, 5.00, 10.00, 20.00, np.inf]
labels = ['< 250 m', '250-1000 m', '1-5 km', '5-10 km', '10-20 km', '> 20 km']
plot_hbar_cat(axs[1], df_ritten, bins, labels, title='Afstand van de ritten')

var = 'snelheid_mean_kph'
bins = [0.00, 5.00, 10.00, 15.00, 20.00, 25.00, np.inf]
labels = ['< 5 km/u', '5-10 km/u', '10-15 km/u', '15-20 km/u', '20-25 km/u', '> 25 km/u']
plot_hbar_cat(axs[2], df_ritten, bins, labels, title='Snelheid van de ritten')

axs[2].axes.get_xaxis().set_label_text("Aantal ritten")


In [ ]:
# Remove datapoints outside of the map area 
#   because it would take a very long time to process large areas.
#   TODO: doe dit op ritniveau (verwijderen ritten deels of geheel buiten de target area)
latlon = {
    'latitude': {'center': mapbox_center['lat'], 'extent': mapbox_extent},
    'longitude': {'center': mapbox_center['lon'], 'extent': mapbox_extent},
}
df = opschonen.filter_lat_lon(df, latlon)


In [ ]:
# Identify a nice colormap.
import plotly.express as px
fig = px.colors.sequential.swatches_continuous()
fig.show()

# => Simple jet may be best


In [ ]:
# Count measurement, rides and users in a hexbin.

import plotly.figure_factory as ff


# agg function to count unique rides and users in the honeycomb.
def n_unique(x):
    return(len(np.unique(x)))

# shared arguments
hexbin_args = {
    'data_frame': df,
    'lat': 'latitude',
    'lon': 'longitude',
    'color_continuous_scale': 'jet',  # bluered
    'min_count': 1,
    'animation_frame': None,
    'width': 1920,
    'height': 1080,
    'opacity': 0.3,
    'zoom': 10,
    'center': mapbox_center,
    'show_original_data': False,
    'labels': {'color': 'N'}, 
    }
layout_args = {
    'mapbox_style': 'open-street-map',
    'coloraxis_showscale': True,
    'mapbox_layers': mapbox_layers,
    'margin': dict(b=0, t=0, l=0, r=0),
    }

# specific arguments
pars = {
    'counts': {
        'color': None,
        'agg_func': None,
        'labels': {'color': 'N'},
        'range_color': [0, 2000],
        },
    'rides': {
        'color': 'rit_id',
        'agg_func': n_unique,
        'labels': {'color': 'ritten'},
        'range_color': [0, 150],
        },
    'users': {
        'color': 'entity_id',
        'agg_func': n_unique,
        'labels': None,
        'range_color': [0, 10],
        },
}

for k, par in pars.items():

    # Replace keys.
    hexbin_args = {**hexbin_args, **par}

    # Plot the data for each month.
    hexbin_args['title'] = period_id
    hexbin_args['nx_hexagon'] = np.ceil(
        (df['longitude'].max() - df['longitude'].min()) / hexagon_size,
        ).astype('int')
    fig = ff.create_hexbin_mapbox(**hexbin_args)
    fig.update_layout(**layout_args)
    fig.update_coloraxes(colorbar_tickfont_size=35)

    # Save image
    filestem = f'{period_id}_hexbin_{k}'
    output_stem = Path(output_directory, filestem)
    fig.write_html(f"{output_stem}.html")
    fig.write_image(f"{output_stem}.png")
    fig.write_image(f"{output_stem}.pdf")
